# DATA 608 Story 1
Title: Infrastructure Investment & Jobs Act Funding Allocation

Yina Qiao

## Intro:
In the exploration of the Infrastructure Investment and Jobs Act funding allocations, I delve into the complexities of equitable distribution and the subtle interplay of politics. The goal is to understand whether the funding reaches states and territories in a manner that reflects their population and to consider how the political landscape might influence these decisions.
## Audience:
This work is intended for a broad audience, from policymakers and civic leaders to academics and engaged citizens, in the hope of contributing thoughtfully to the ongoing discussion on building a more equitable and responsive infrastructure for all.






### Data
I've harnessed datasets below for the analysis, carefully preprocessing them in Excel and hosting on GitHub for accessibility. Their sources in the reference section

1. Data on the present allocation of the Infrastructure Investment and Jobs Act funding by State and Territory (funding data)
2. Estimated population data for each state and territory (population data)
3. Official election results of the 2020 Presidential election (election data)

In [63]:
# @title
import pandas as pd
import matplotlib.pyplot as plt
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import seaborn as sns

# Function to read Excel file from GitHub
def read_excel_from_github(url):
    response = requests.get(url)
    data = BytesIO(response.content)
    return pd.read_excel(data)

# URLs of the raw Excel files on GitHub
funding_url = 'https://github.com/yinaS1234/data-608/raw/main/IIJA%20FUNDING%20AS%20OF%20MARCH%202023(1).xlsx'
population_url = 'https://github.com/yinaS1234/data-608/raw/main/Current%20Population%20by%20State.xlsx'
election_results_url = 'https://github.com/yinaS1234/data-608/raw/main/2020_Presidential_Election_Results.xlsx'



# Read the datasets
funding = read_excel_from_github(funding_url)
population = read_excel_from_github(population_url)
election_results = read_excel_from_github(election_results_url)
# Standardize state/territory
funding['State/Territory'] = funding['State/Territory'].str.title()
population['State/Territory'] = population['State/Territory'].str.title()
election_results['State/Territory'] = election_results['State/Territory'].str.title()

# Merge
merged_data = pd.merge(election_results, population, on='State/Territory', how='outer')
final_data = pd.merge(merged_data, funding, on='State/Territory', how='outer')

# Calculate per capita funding allocation (in dollars)
final_data['Per Capita Funding'] = (final_data['Total (Billions)'] * 1e9) / final_data['Population 2023']

# Replace any potential NaN values with zeros for further analysis
final_data.fillna(0, inplace=True)

# Create a column 'State_Abbrev' for easy visual.
state_abbreviations = {
    **{  # Standard states
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
        'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
        'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
        'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
        'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
        'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
        'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
        'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    },
    **{  # Territories and special cases
        'District Of Columbia': 'DC', 'Puerto Rico': 'PR', 'Guam': 'GU', 'Northern Mariana Islands': 'MP',
        'US Virgin Islands': 'VI', 'American Samoa': 'AS', 'Tribal Communities': 'TC'
    }
}
state_abbreviations['Us Virgin Islands'] = 'VI'
final_data['State_Abbrev'] = final_data['State/Territory'].map(state_abbreviations)

### Analysis and Visual

In [64]:
# @title
choropleth_data['Per Capita Funding (in $1k)'] = round(choropleth_data['Per Capita Funding'] / 1000, 2)


fig_choropleth_funding = px.choropleth(
    choropleth_data,
    locations='State_Abbrev',
    locationmode="USA-states",
    color='Per Capita Funding (in $1k)',
    scope="usa",
    color_continuous_scale="Viridis",
    labels={'Per Capita Funding (in $1k)': 'Per Capita Funding ($1k)', 'State_Abbrev': 'State'},
    title='Per Capita Funding by State'
)

fig_choropleth_funding.update_layout(
    title={
        'text': "Per Capita Funding by State",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24
        )
    }
)


fig_choropleth_funding = px.choropleth(
    choropleth_data,
    locations='State_Abbrev',
    locationmode="USA-states",
    color='Per Capita Funding (in $1k)',
    scope="usa",
    color_continuous_scale="Teal",  # Changed to a Teal color scale
    labels={'Per Capita Funding (in $1k)': 'Per Capita Funding ($1k)', 'State_Abbrev': 'State'},
    title='Per Capita Funding by State'
)

fig_choropleth_funding.update_layout(
    title={
        'text': "Per Capita Funding by State",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24)
    }
)




fig_choropleth_funding.show()


final_data['Winning Party'] = final_data.apply(lambda row: 'Biden' if row['Biden Electoral Votes'] > 0 else 'Trump', axis=1)
political_choropleth_data = final_data[['State_Abbrev', 'Winning Party']]


fig_choropleth_political = px.choropleth(
    political_choropleth_data,
    locations='State_Abbrev',
    locationmode="USA-states",
    color='Winning Party',
    scope="usa",
    color_discrete_map={'Biden': 'blue', 'Trump': 'red'},
    labels={'Winning Party': 'Winning Party', 'State_Abbrev': 'State'},
    title='Political Alignment by State (2020 Presidential Election)'
)
fig_choropleth_political.update_layout(
    title={
        'text': "Per Capita Funding by State",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=24
        )
    }
)



fig_choropleth_political.show()


Result:

The per capita funding map provides a clear visual insight into how funding is distributed relative to population size, enabling viewers to easily identify states that may be receiving more or less funding than might be expected based on their populations.
The political alignment map offers a straightforward visualization of the political landscape, facilitating an analysis of whether there's a correlation between a state's political leaning and the funding it received.
We can cross-rerefence the funding equtiy with political alignment to drive insights.

Conclusion

1. Is the allocation equitable based on the population of each of the States and Territories, or is bias apparent?


*  Ideally, funding per person should be similar everywhere, but it's not.
*  States like Alaska, Wyoming, North Dakota, and Montana, despite their lower populations, receive high funding per capita, possibly due to their vast land areas and unique infrastructural demands.


2. Does the allocation favor the political interests of the Biden administration?


* The allocation of high per capita funding to states that voted for Trump in 2020 suggests that funding decisions do not specifically favor the political interests of the Biden administration.




Reference:

https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html

https://www.archives.gov/electoral-college/2020

https://en.wikipedia.org/wiki/2020_United_States_presidential_election